### Project2

In [17]:
# import libraries!

import pandas as pd
import numpy as np

In [40]:
# read the dataset
data = pd.read_excel("WA_Fn-UseC_-Telco-Customer-Churn.xlsx", na_values=[" "])

print("Dimension of the dataset: {}".format(data.shape))

# print the first 5 rows to see what the dataset is about
data.head()

#Changing from int to categorical
data['SeniorCitizen']=pd.Categorical(data['SeniorCitizen']) 

# deleting the customerID features since it doesn't give any information about the churn..
del data["customerID"]
data = data.dropna() # drop NA values

# splitting the dataset to numeric and non-numeric 
numeric = data.select_dtypes(include = ['float64','int64'])
non_numeric = data.select_dtypes(exclude = ['float64','int64'])

Dimension of the dataset: (7043, 21)


The dataset has 7043 samples and 21 features. The features includes demographic information about the costumer like gender, age, if he/she has partner and dependents, type of services that the customer has signed up for. Also, account information like how long has been a costumer, contract, payment method, paperless billing, monthly charges and total chargers. Lastly, we have the churn feature which refers to the customers who left the company in the past month.